

---




---




![picture](https://i.ibb.co/LJ8y7dc/bds-banner.png)



---



---



**[Summary Link](https://docs.google.com/document/d/1SzfgMYKljHrZhEYt3xxOaxkHIdYaQo8USy-Z1glXyzQ/edit?usp=sharing)**

**[Slide Deck Link](https://docs.google.com/presentation/d/1hyKoFc4Ejmp0SKVgX1DjLMUyONqiXWa0eQogl8XGnk0/edit?usp=sharing)**

# 1. Business Problem:



**To understand Employee Attrition**

# 2. Data Description:

**Data Source :** [Kaggle](https://www.kaggle.com/dineshsagar66/hr-analytics-people-management)

The data has 14999 rows & 10 columns. Each row is for a different employee.

**Column Details :**

satisfaction_level: Satisfaction level score between 0 and 1

last_evaluation: Last evaluation score given to the employee

number_project: Number of projects worked on per year

average_montly_hours: Average monthly working hours

time_spend_company: Time spent in the company in years

Work_accident: Whether they have had a work accident in the last 2 years

promotion_last_5years: Whether they have had a promotion in the last 5 years

department: Department the employee is working in

salary: salary of the employee (low, medium, high)

left: Whether the employee has left or is still working

# 3. Spark Set-Up:

In [ ]:
!apt-get update 
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

!wget -q http://apache.osuosl.org/spark/spark-3.0.0/spark-3.0.0-bin-hadoop2.7.tgz

!tar xf spark-3.0.0-bin-hadoop2.7.tgz 
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop2.7"

import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:11 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Hit:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Reading package lists... Done


**Explanation :**

The latest version of spark is getting downloaded and installed in this colab machine

# 4. Data Loading:

In [ ]:
from google.colab import files
data_to_load = files.upload()

Saving employee_data.csv to employee_data.csv
Saving employee_headers.csv to employee_headers.csv


**Explanation :**

Uploading data files to colab from local machine

In [ ]:
raw_data = sc.textFile('employee_data.csv')
headers = sc.textFile('employee_headers.csv')

**Explanation :**

1. Storing the data as a rdd to variable 'raw_data'

2. Storing the header names to variable 'headers'

In [ ]:
raw_data.take(5)

['0.41,0.46,2,128,3,0,0,accounting,low,1',
 '0.65,0.76,4,171,4,0,0,technical,low,0',
 '0.94,0.52,3,273,3,0,0,product_mng,high,0',
 '0.92,0.99,4,245,4,0,0,IT,medium,0',
 '0.43,0.46,2,126,3,0,0,sales,medium,1']

**Explanation :**

Having a look at the first 5 data rows

In [ ]:
headers.collect()

['satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,promotion_last_5years,department,salary,left']

**Explanation :**

Having a look at the header names

# 5. Exploratory Data Analysis:

## 5.a. Data Preparation for doing Statistical Analysis:

In [ ]:
data = raw_data.map(lambda x: x.split(","))

departments = data.map(lambda x: x[7]).distinct().collect()
salary = data.map(lambda x: x[8]).distinct().collect()

**Explanation :**

Since 'departments' and 'salary' are categorical, the distinct values of these features are stored separately which will be used for Label Encoding


In [ ]:
import numpy as np

def parse_interaction(line):
    
    # Label Encoding 'salary' since low, medium, high is in an order which can be replaced by 0, 1, 2 respectively 
    try:
        line[8] = salary.index(line[8])
    except:
        line[8] = len(salary)
    
    # 7th index is department which is categorical, hence not including it
    clean_line = [item for i,item in enumerate(line) if i not in [7]]

    return np.array([float(x) for x in clean_line])

numeric_data = data.map(parse_interaction)

**Explanation :**
1. First imported the library numpy as numpy arrray will be used.

2. The salary feature has been Label Encoded, since 'low', 'medium', 'high' is in an order which can be replaced by 0, 1, 2 respectively and we can further perform statistics on this feature as well.

3. This function will finally return an array with all the numeric features, including label encoded converted salary, as an numpy array. The department feature is excluded .

4. At the end, the data is being mapped with the created function and it is stored in 'numeric_data'.

## 5.b. Correlation Analysis: 

In [ ]:
import pandas as pd
from pyspark.mllib.stat import Statistics

correlation_matrix = Statistics.corr(numeric_data, method="pearson")


col_names = ['satisfaction_level','last_evaluation','number_project','average_montly_hours', \
              'time_spend_company','Work_accident','promotion_last_5years','salary','left']

corr_df = pd.DataFrame(correlation_matrix, index=col_names, columns=col_names)

print('Correlation among  the variables taking the complete data : \n')

corr_df

Correlation among  the variables taking the complete data : 



,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,promotion_last_5years,salary,left
satisfaction_level,1.000000,0.105021,-0.142970,-0.020048,-0.100866,0.058697,0.025605,0.041029,-0.388375
last_evaluation,0.105021,1.000000,0.349333,0.339742,0.131591,-0.007104,-0.008684,0.000476,0.006567
number_project,-0.142970,0.349333,1.000000,0.417211,0.196786,-0.004741,-0.006064,0.005129,0.023787
average_montly_hours,-0.020048,0.339742,0.417211,1.000000,0.127755,-0.010143,-0.003544,0.003141,0.071287
time_spend_company,-0.100866,0.131591,0.196786,0.127755,1.000000,0.002120,0.067433,0.030444,0.144822
Work_accident,0.058697,-0.007104,-0.004741,-0.010143,0.002120,1.000000,0.039245,0.004522,-0.154622
promotion_last_5years,0.025605,-0.008684,-0.006064,-0.003544,0.067433,0.039245,1.000000,0.064524,-0.061788
salary,0.041029,0.000476,0.005129,0.003141,0.030444,0.004522,0.064524,1.000000,-0.106071
left,-0.388375,0.006567,0.023787,0.071287,0.144822,-0.154622,-0.061788,-0.106071,1.000000


**Explanation :**

Creating a dataframe of correlation between all the features and the label column.


**Observations :**

1. There is no high correaltion among the features.

2. There is moderate negative correaltion between left and satifaction_level (-0.39).

3. There is moderate positive correaltion between number_project with last_evaluation(0.35) and avarage_monthly_hours (0.41)

## 5.c. Correaltion Analysis (Label Wise):

In [ ]:
def parse_interaction_with_key(line):
      
    try:
        line[8] = salary.index(line[8])
    except:
        line[8] = len(salary)
    
    clean_line = [item for i,item in enumerate(line) if i not in [7]]
    return (line[9], np.array([float(x) for x in clean_line]))

label_numeric_data = data.map(parse_interaction_with_key)

**Explanation :**

1. Like it was done for the previous function (parse_interaction), a new function is defined and salary is label encoded.

2. This returns key value pair of Label along with the other variables.

3. The function is applied to data and is stored in a variable named 'label_numeric_data'. 






**To dig deeper, the data of those who have left must be observed:**

In [ ]:
# filtering basis values of left. left==1 means person has quitted the company
quitted_data = numeric_data.filter(lambda x: x[8]==1)

quitted_corr_matrix = Statistics.corr(quitted_data, method="pearson")

quitted_corr_df = pd.DataFrame(quitted_corr_matrix, index=col_names, columns=col_names)

print('Correlation among features for those who have left the company : \n')
quitted_corr_df.iloc[:8, :8] # excluding the column 'left' as all values of left are 1

Correlation among features for those who have left the company : 



,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,promotion_last_5years,salary
satisfaction_level,1.000000,0.182295,-0.227113,-0.084117,0.446440,0.005864,0.005516,-0.005019
last_evaluation,0.182295,1.000000,0.804155,0.830336,0.783873,-0.001875,-0.047992,0.016157
number_project,-0.227113,0.804155,1.000000,0.875194,0.601989,-0.008401,-0.032301,0.018866
average_montly_hours,-0.084117,0.830336,0.875194,1.000000,0.664011,-0.009048,-0.035475,0.021938
time_spend_company,0.446440,0.783873,0.601989,0.664011,1.000000,0.005222,-0.045888,0.006993
Work_accident,0.005864,-0.001875,-0.008401,-0.009048,0.005222,1.000000,0.056214,0.026766
promotion_last_5years,0.005516,-0.047992,-0.032301,-0.035475,-0.045888,0.056214,1.000000,-0.017865
salary,-0.005019,0.016157,0.018866,0.021938,0.006993,0.026766,-0.017865,1.000000


**Explanation :**

1. The data has been filtered and only those who have quitted the job are taken.

2. Correaltion matrix is formed

**Observations :**

1. Here we see high correaltion between many features. 

2. As expected, average monthly hours is highly correlated with number of projects (0.87). The more time an employee gives, the more project he does.

3. Last evaluation score is highly correlated with  average monthly hours (0.83) and number of projects (0.80). This is expected as high evaluation score is given to employees who work on more projects by spending more time every month.

## 5.d. Analysis using pyspark.sql dataframes:

In [ ]:
from pyspark.sql import Row

# mapping the RDD to a DF for further analysis:
df = data.map(lambda line: Row(satisfaction_level=line[0], 
                              last_evaluation=line[1], 
                              number_project=line[2],
                              average_montly_hours=line[3],
                              time_spend_company=line[4],
                              Work_accident=line[5], 
                              promotion_last_5years=line[6],
                              department=line[7],
                              salary=line[8],
                              left=line[9])).toDF()                

**Explanation :**

The data is converted to a dataframe for further analysis.

In [ ]:
print('Head of the Data : \n')
df.show(10) # looking at the first 10 rows

Head of the Data : 

+------------------+---------------+--------------+--------------------+------------------+-------------+---------------------+-----------+------+----+
|satisfaction_level|last_evaluation|number_project|average_montly_hours|time_spend_company|Work_accident|promotion_last_5years| department|salary|left|
+------------------+---------------+--------------+--------------------+------------------+-------------+---------------------+-----------+------+----+
|              0.41|           0.46|             2|                 128|                 3|            0|                    0| accounting|   low|   1|
|              0.65|           0.76|             4|                 171|                 4|            0|                    0|  technical|   low|   0|
|              0.94|           0.52|             3|                 273|                 3|            0|                    0|product_mng|  high|   0|
|              0.92|           0.99|             4|                

In [ ]:
print(' Statistical summary of the features for the complete data: \n')
df.describe().show()

 Statistical summary of the features for the complete data: 

+-------+-------------------+-------------------+------------------+--------------------+------------------+------------------+---------------------+----------+------+-------------------+
|summary| satisfaction_level|    last_evaluation|    number_project|average_montly_hours|time_spend_company|     Work_accident|promotion_last_5years|department|salary|               left|
+-------+-------------------+-------------------+------------------+--------------------+------------------+------------------+---------------------+----------+------+-------------------+
|  count|              14999|              14999|             14999|               14999|             14999|             14999|                14999|     14999| 14999|              14999|
|   mean| 0.6128335222348125| 0.7161017401160044|  3.80305353690246|   201.0503366891126| 3.498233215547703|0.1446096406427095| 0.021268084538969265|      null|  null| 0.2380825388359224

**Explanation :**

Looking at the statistical summary of columns, taking both working and quitted employee's data


**Observations :**

1. The employees are 61% satisfied with their jobs on an average.

2. Out of 1 they have mean performance score of 0.71 .

3. On an average, an employee worked on 3 to 4 projects (3.8 mean).

4. The employees take 201 hours to do their jobs per month.

5. An average employee worked almost 3.5 years in the company. The minimum number of working years among the employees is 2 and the maximum is 10.

6. Only 2.1% of employees were promoted in the last 5 years.

7. Employee churn rate is 23.81%




**A deeper analysis is required by splitting the summary basis the labels:**

In [ ]:
print('Statistical summary for those who are still working: \n')
df.where('left==0').describe().show()

print('\n','=x'*85, '\n \n')

print('Statistical summary for those who have left: \n')
df.where('left==1').describe().show()


Statistical summary for those who are still working: 

+-------+-------------------+-------------------+------------------+--------------------+------------------+-------------------+---------------------+----------+------+-----+
|summary| satisfaction_level|    last_evaluation|    number_project|average_montly_hours|time_spend_company|      Work_accident|promotion_last_5years|department|salary| left|
+-------+-------------------+-------------------+------------------+--------------------+------------------+-------------------+---------------------+----------+------+-----+
|  count|              11428|              11428|             11428|               11428|             11428|              11428|                11428|     11428| 11428|11428|
|   mean| 0.6668095904795224| 0.7154733986699311| 3.786664333216661|   199.0602030101505|3.3800315015750786|0.17500875043752187| 0.026251312565628283|      null|  null|  0.0|
| stddev|0.21710375451971922|0.16200495648503893|0.9798837648210726|  

**Explanation :**

The data is filtered basis the labels and then label-wise statistical summaries for the variables are printed.

**Observations :**

1. The working employees are 66% satisfied but the quitted ones were only 44% satisfied, on an average.

2. Out of 1, the mean performance score of quitted ones (0.718) are slightly better than the working employees (0.715)

3. On an average, the ones who have quitted, had worked on more projects (3.85) , than the employees who are still working(3.78).

4. The quitted ones had also spent more time monthly working , and, they were working for a longer period than the still working ones.

6. Among the quitted employees, a mere 0.5% of employees were promoted in the last 5 years in comparison to 2.6% among the working ones.

7. It is clearly observed that quitted employees worked more but were promoted less and were less satisfied.



In [ ]:
print('Salary-wise number of employees those who have left and those who are still working : \n')

df.groupBy("salary",'left')\
.count().sort('salary',"left").show()

Salary-wise number of employees those who have left and those who are still working : 

+------+----+-----+
|salary|left|count|
+------+----+-----+
|  high|   0| 1155|
|  high|   1|   82|
|   low|   0| 5144|
|   low|   1| 2172|
|medium|   0| 5129|
|medium|   1| 1317|
+------+----+-----+



**Observations :**
 
 1. There is high churn rate among the lower paid employees. Close to 30% low paid employees have left

 2. The rate is high among medium paid employees as well, which is about 20%

 3. The high paid employees have the least churn rate of about 6%

In [ ]:
print('Department-wise number of employees those who have left and those who are still working : \n')

df.groupBy("department",'left')\
.count().sort("department",'left').show()

Department-wise number of employees those who have left and those who are still working : 

+-----------+----+-----+
| department|left|count|
+-----------+----+-----+
|         IT|   0|  954|
|         IT|   1|  273|
|      RandD|   0|  666|
|      RandD|   1|  121|
| accounting|   0|  563|
| accounting|   1|  204|
|         hr|   0|  524|
|         hr|   1|  215|
| management|   0|  539|
| management|   1|   91|
|  marketing|   0|  655|
|  marketing|   1|  203|
|product_mng|   0|  704|
|product_mng|   1|  198|
|      sales|   0| 3126|
|      sales|   1| 1014|
|    support|   0| 1674|
|    support|   1|  555|
|  technical|   0| 2023|
|  technical|   1|  697|
+-----------+----+-----+




**Observations :**
 
 1. The churn rate is more or less similar across all the departments except RandD and management.
 
 2. The churn rate is high in the HR department, which is as high as 28.5%

 3. Among the RandD and management the rate is 15.37% and 14.44% respectively which is lower than the other departments.


In [ ]:
print('Count of employees those who have left and those who are still working along with their number of years working: \n')


df.groupBy("time_spend_company",'left')\
.count().sort('left',"time_spend_company").show()

Count of employees those who have left and those who are still working along with their number of years working: 

+------------------+----+-----+
|time_spend_company|left|count|
+------------------+----+-----+
|                10|   0|  214|
|                 2|   0| 3191|
|                 3|   0| 4857|
|                 4|   0| 1667|
|                 5|   0|  640|
|                 6|   0|  509|
|                 7|   0|  188|
|                 8|   0|  162|
|                 2|   1|   53|
|                 3|   1| 1586|
|                 4|   1|  890|
|                 5|   1|  833|
|                 6|   1|  209|
+------------------+----+-----+



**Observations :**

1. A very high number of employees have left after working for 3 to 6 years.

2. The retention rate is 100% for employees those have worked for 7 or more years.

In [ ]:
print('Years spent in company along with the average of features for those who are Working: \n')

df.where('left==0').groupBy("time_spend_company")\
.agg({"satisfaction_level":"avg",\
      "last_evaluation":"avg",\
      'number_project':"avg", \
      'average_montly_hours':"avg",\
      'Work_accident':"avg"})\
      .sort('time_spend_company')\
.show()

print('\n','=x'*65, '\n \n')

print('Years spent in company along with the average of features for those who have Left: \n')


df.where('left==1').groupBy("time_spend_company")\
.agg({"satisfaction_level":"avg",\
      "last_evaluation":"avg",\
      'number_project':"avg", \
      'average_montly_hours':"avg",\
      'Work_accident':"avg"})\
      .sort('time_spend_company')\
.show()

Years spent in company along with the average of features for those who are Working: 

+------------------+--------------------+-------------------+-----------------------+-------------------------+-------------------+
|time_spend_company|avg(last_evaluation)| avg(Work_accident)|avg(satisfaction_level)|avg(average_montly_hours)|avg(number_project)|
+------------------+--------------------+-------------------+-----------------------+-------------------------+-------------------+
|                10|  0.7314953271028035| 0.2336448598130841|     0.6553271028037382|       199.22429906542055|  3.682242990654206|
|                 2|  0.7170040739580064|0.17455343152616734|     0.6999905985584463|       199.95643998746473| 3.6812911313068004|
|                 3|  0.7172781552398608|0.16903438336421658|     0.6963804817788761|       199.45151327980236| 3.7294626312538606|
|                 4|  0.7219736052789446|0.16376724655068986|     0.6386742651469709|        198.8122375524895|  3.916016

**Observations :**
1. Interestingly, the performance score of quitted employees, those who have worked betwwen 4 to 6 years is very high (0.85 to 0.90), even higher than those who are still working.

2. The quitted employees had very few work accidents than those who are working.

3. Among the quitted ones, the satisfaction level of those worked between 2 to 4 years is pretty low, as low as 0.14 to 0.52.

4. On an average the employees those have spent 4 to 6 years in the company, the quitted employees have spent a lot more time working than the still working ones on a higher number of projects on average per year.

5. So overall this can be observed that the experienced employees , those who are working more and hard and have a very good performance score , haven't been paid attention to by the company. 

6. The employees who have worked more than 7 years, although they have lower satisfacton level (than those who have worked 2 or 3 years and still working) , they tend to stay with the company,this could be because they are used to the company and want to walk on the same known road.


# 6. Predictive Models: 

## 6.a. Data Preparation for building Predictive Models:

In [ ]:
from pyspark.mllib.regression import LabeledPoint
from numpy import array

def create_labeled_point(line_split):
    clean_line_split = line_split[0:9]
    
    # convert departments to numeric categorical variable
    try: 
        clean_line_split[7] = departments.index(clean_line_split[7])
    except:
        clean_line_split[7] = len(departments)
        
    # convert salary to numeric categorical variable
    try:
        clean_line_split[8] = salary.index(clean_line_split[8])
    except:
        clean_line_split[8] = len(salary)

        
    return LabeledPoint(line_split[9], array([float(x) for x in clean_line_split]))

labeled_data = data.map(create_labeled_point)

**Explanation :**

1. The objective here is to prepare the data to run machine learning models

2. A function is defined which will convert the categorical variable department and salary to numbers.

3. The function will return a key value pair of the label along with it's features

4. At the end, this function is applied to the data and is stored in variable named 'labeled_data'.

In [ ]:
# spliting the labeled data to train and test set

train_data, test_data = labeled_data.randomSplit([.8,.2],seed=123)

## 6.b. Decision Tree Model:

**Model Building:**

In [ ]:
from pyspark.mllib.tree import DecisionTree, DecisionTreeModel
from time import time

# building the model:
t0 = time()
tree_model = DecisionTree.trainClassifier(train_data, numClasses=2, 
                                          categoricalFeaturesInfo={7: len(departments), 8: len(salary)},
                                          impurity='gini', maxDepth=4, maxBins=100)
tt = time() - t0

print("Classifier trained in {} seconds".format(round(tt,3)))

Classifier trained in 3.081 seconds


**Explanation :**

The Decision tree classifier model is created and trained using MLlib and among the parameters, a parameter is passed telling the model that the features 7 and 8 are categorical, along with the number of unique values in thaose features

**Model Evaluation:**

In [ ]:
predictions = tree_model.predict(test_data.map(lambda p: p.features))
labels_and_preds = test_data.map(lambda p: p.label).zip(predictions)

t0 = time()
test_accuracy = labels_and_preds.filter(lambda vp: vp[0] == vp[1]).count() / float(test_data.count())
tt = time() - t0

print("Prediction made in {} seconds. Test accuracy is {}".format(round(tt,3), round(test_accuracy,4)))

Prediction made in 1.54 seconds. Test accuracy is 0.9725


**Model Visualisation:**

In [ ]:
print("Learned classification tree model:")
print(tree_model.toDebugString())

Learned classification tree model:
DecisionTreeModel classifier of depth 4 with 25 nodes
  If (feature 0 <= 0.46499999999999997)
   If (feature 2 <= 2.5)
    If (feature 1 <= 0.575)
     If (feature 1 <= 0.445)
      Predict: 0.0
     Else (feature 1 > 0.445)
      Predict: 1.0
    Else (feature 1 > 0.575)
     If (feature 3 <= 278.5)
      Predict: 0.0
     Else (feature 3 > 278.5)
      Predict: 1.0
   Else (feature 2 > 2.5)
    If (feature 0 <= 0.11499999999999999)
     Predict: 1.0
    Else (feature 0 > 0.11499999999999999)
     If (feature 2 <= 6.5)
      Predict: 0.0
     Else (feature 2 > 6.5)
      Predict: 1.0
  Else (feature 0 > 0.46499999999999997)
   If (feature 4 <= 4.5)
    If (feature 3 <= 287.5)
     If (feature 2 <= 6.5)
      Predict: 0.0
     Else (feature 2 > 6.5)
      Predict: 1.0
    Else (feature 3 > 287.5)
     Predict: 1.0
   Else (feature 4 > 4.5)
    If (feature 1 <= 0.815)
     Predict: 0.0
    Else (feature 1 > 0.815)
     If (feature 3 <= 216.5)
      Pre

**Observations :**


1. Satisfaction level (feature 0) seems to be the most important variable. If employee satisfaction level is above 0.46 , they’re much more likely to stay.

2. If employees have low satisfaction, the number of projects (feature 2) becomes crucial to them. If the employees have more projects, they’re more likely to remain. If they have fewer projects, they leave.

3. If employees are satisfied ( satisfaction_level > 0.46), and they have been at the company for less than 4.5 years (feature 4), and scored over 81% (feature 1) in their last evaluation, they are very likely to leave.

4. So, by visualising the Decision Tree spliting criterias, we can see that the features 0,1,2 and 4 are the important features.

5. They are namely : satisfaction_level, last_evaluation, number_project and time_spend_company

## 6.c. Decison Tree Model (Reduced Features):

**Data Preparation with reduced features:**

In [ ]:
def create_labeled_point_minimal(line_split):
    # leave_out = [9]
    clean_line_split = line_split[0:3] + line_split[4:5]
  
        
    return LabeledPoint(line_split[9], array([float(x) for x in clean_line_split]))

labeled_data_minimal = data.map(create_labeled_point_minimal)
train_data_minimal, test_data_minimal = labeled_data_minimal.randomSplit([.8,.2],seed=123)

**Explanation :**

Here a new function is defined that return key value pair of label along with the 4 feature values : satisfaction_level, last_evaluation, number_project and time_spend_company

**Model Building with Reduced Features:**

In [ ]:
t0 = time()
tree_model_minimal = DecisionTree.trainClassifier(train_data_minimal, numClasses=2, 
                                          categoricalFeaturesInfo={},
                                          impurity='gini', maxDepth=4, maxBins=100)
tt = time() - t0

print("Classifier trained in {} seconds".format(round(tt,3)))

Classifier trained in 1.665 seconds


**Explanation :**
Buildig a new decision tree model, with only the 4 features

**Model Evaluation (Minimal Model):**

In [ ]:
predictions_minimal = tree_model_minimal.predict(test_data_minimal.map(lambda p: p.features))
labels_and_preds_minimal = test_data_minimal.map(lambda p: p.label).zip(predictions_minimal)

t0 = time()
test_accuracy = labels_and_preds_minimal.filter(lambda vp: vp[0] == vp[1]).count() / float(test_data_minimal.count())
tt = time() - t0

print("Prediction made in {} seconds. Test accuracy is {}".format(round(tt,3), round(test_accuracy,4)))

Prediction made in 1.066 seconds. Test accuracy is 0.9715


**Observation :**

This can be observed here that instead of 9 predictor features, using just the 4 features is resulting in almost equal accuracy.

In [ ]:
print('Minimal Decison tree :')

print(tree_model_minimal.toDebugString())

Minimal Decison tree :
DecisionTreeModel classifier of depth 4 with 21 nodes
  If (feature 0 <= 0.46499999999999997)
   If (feature 2 <= 2.5)
    If (feature 1 <= 0.575)
     If (feature 1 <= 0.445)
      Predict: 0.0
     Else (feature 1 > 0.445)
      Predict: 1.0
    Else (feature 1 > 0.575)
     Predict: 0.0
   Else (feature 2 > 2.5)
    If (feature 0 <= 0.11499999999999999)
     Predict: 1.0
    Else (feature 0 > 0.11499999999999999)
     If (feature 2 <= 6.5)
      Predict: 0.0
     Else (feature 2 > 6.5)
      Predict: 1.0
  Else (feature 0 > 0.46499999999999997)
   If (feature 3 <= 4.5)
    If (feature 2 <= 6.5)
     Predict: 0.0
    Else (feature 2 > 6.5)
     Predict: 1.0
   Else (feature 3 > 4.5)
    If (feature 1 <= 0.815)
     Predict: 0.0
    Else (feature 1 > 0.815)
     If (feature 3 <= 6.5)
      Predict: 1.0
     Else (feature 3 > 6.5)
      Predict: 0.0



## 6.d. Logestic Regression Model:

**Hypothesis Tesing to check feature significance for building logistic regression model:**

In [ ]:
chi = Statistics.chiSqTest(labeled_data)
pd.set_option('display.max_colwidth', 30)

feature_names = ['satisfaction_level','last_evaluation','number_project','average_montly_hours',
                 'time_spend_company','Work_accident','promotion_last_5years','salary','left']

records = [(result.statistic, result.pValue) for result in chi]

chi_df = pd.DataFrame(data=records, index= feature_names, columns=["Statistic","p-value"])

chi_df.iloc[:8, :]

,Statistic,p-value
satisfaction_level,7937.744226,0.000000e+00
last_evaluation,2534.841922,0.000000e+00
number_project,5373.586480,0.000000e+00
average_montly_hours,3623.053923,0.000000e+00
time_spend_company,2110.079505,0.000000e+00
Work_accident,358.593836,0.000000e+00
promotion_last_5years,57.262734,3.808065e-14
salary,86.825474,6.994405e-15


**Observation :**

This can be observed from the chi square test that all the variables are independent and are significant and thus all the variables should be included in our logistic regression model.

**Logistic Regression Model Building:**

In [ ]:
from pyspark.mllib.classification import LogisticRegressionWithLBFGS

t0 = time()
logit_model = LogisticRegressionWithLBFGS.train(train_data)
tt = time() - t0

print("Classifier trained in {} seconds".format(round(tt,3)))

Classifier trained in 5.832 seconds


**Model Evaluation:**

In [ ]:
LR_labels_and_preds = test_data.map(lambda p: (p.label, logit_model.predict(p.features)))

t0 = time()
LR_test_accuracy = LR_labels_and_preds.filter(lambda p: p[0] == p[1]).count() / float(test_data.count())
tt = time() - t0

print("Prediction made in {} seconds. Test accuracy is {}".format(round(tt,3), round(LR_test_accuracy,4)))

Prediction made in 0.389 seconds. Test accuracy is 0.7729


**Observation :**

The decision tree model here performs much better than the logistic regression model for this data. Hence, for this data Decision Tree Model is better for doing predictive analytics.

# 7. Conclusion:

* Through the analysis, it is found that the satisfaction level is the key deciding factor to leave the job, hence managing the level of satisfaction is important to keep employees with the firm.

*	One of the ways to increase employee satisfaction can be through increasing the pay, especially among the lower paid employees.

*	If employee’s satisfaction level is higher than 0.46, they do not quit in a short period (4.5 years).

*	However, if satisfaction level is lower than 0.46, then other factors become important for the resignation decision, such as number of projects they are working on or the working hours. So, if an employee’s satisfaction is low and he/she is overworked, then there is a high chance of resignation.

*	One interesting finding was that, if an employee is working in the company for more than 4 years and have very high evaluation scores and are spending more time working, they tend to leave the company. If an employee is passionate about his work and thinks that they aren’t paid what they deserve, then they might leave the company. Salary hikes and increasing number of promotions becomes crucial for retention for such employees.

*	These results can be further brain-stormed and effective methods can be implemented by the firm in order to retain top talents and keep them from leaving.
